## À FAIRE : 
- S'occuper des accents (demander à Houssam)

In [1]:
import pandas as pd
from sklearn.utils import shuffle

import tensorflow as tf

### Récupération de l'ensemble de données

In [2]:
path = "/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/datasets/Tweeter/french_tweets.csv"

df = pd.read_csv(path)
df = shuffle(df)

In [3]:
df_size = 1000
df = df[:df_size]
df

,label,text
1199141,1,Productif mais pas aussi calme que id'a a aimé...
1082000,1,"Je n'ai pas ... non, je t'ai eu la chose en ti..."
1357253,1,Nous utilisons chipotle tabasco pour les ailes...
693626,0,Je veux regarder me traîner vers l'enfer jaloux
1368336,1,Rien ne réchauffe mon éloge que de s'asseoir d...
...,...,...
1141882,1,Je suis juste revenu de travailler. J'ai vu un...
746381,0,Idk vraiment. Le safari s'arrête également bea...
1433638,1,Indépendamment de ce qui se passe à partir de ...
460310,0,"Appareil photo, soyez d'accord."


In [4]:
targets = df['label']
targets.head()

1199141    1
1082000    1
1357253    1
693626     0
1368336    1
Name: label, dtype: int64

In [5]:
text_sequences = df['text']
text_sequences

1199141    Productif mais pas aussi calme que id'a a aimé...
1082000    Je n'ai pas ... non, je t'ai eu la chose en ti...
1357253    Nous utilisons chipotle tabasco pour les ailes...
693626       Je veux regarder me traîner vers l'enfer jaloux
1368336    Rien ne réchauffe mon éloge que de s'asseoir d...
                                 ...                        
1141882    Je suis juste revenu de travailler. J'ai vu un...
746381     Idk vraiment. Le safari s'arrête également bea...
1433638    Indépendamment de ce qui se passe à partir de ...
460310                       Appareil photo, soyez d'accord.
1427179    Aaaaaaaaawawwwwwwwwwwwwww high school sweethea...
Name: text, Length: 1000, dtype: object

In [6]:
ds = tf.data.Dataset.from_tensor_slices((text_sequences, targets))
len(ds)

2022-08-03 15:39:12.708118: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


1000

In [7]:
b_ds = ds.batch(32)
iterator_ds = iter(b_ds)

In [8]:
text, label = next(iterator_ds)
text.shape, label.shape

(TensorShape([32]), TensorShape([32]))

### Fonctions principales

In [9]:
def build_datasets(df, ds_size, batch_size, vocab_size, max_length):
    print("------------")
    
    def create_ds(df, size):
        shuffled_df = shuffle(df)[:size]
        text_seq = shuffled_df['text']
        target_seq = shuffled_df['label']
        ds = tf.data.Dataset.from_tensor_slices((text_seq, target_seq))
        
        return ds
    
    ds = create_ds(df, ds_size)
    ds_size = len(ds)    
    print(f" Ensemble de données créé, taille : {ds_size}")   
    print("------------")


    train_size = int(0.7 * ds_size)
    val_size = int(0.15 * ds_size)
    print(f" Taille des ensembles de données : {train_size}, {val_size}")
    
    ds.shuffle(1)

    str_train_ds = ds.take(train_size).batch(batch_size)
    str_val_ds = ds.skip(train_size).take(val_size).batch(batch_size)
    str_test_ds = ds.skip(train_size + val_size).batch(batch_size)

    print("Fin du chargement des bases de données")
    print(len(str_train_ds) * batch_size, len(str_val_ds) * batch_size, len(str_test_ds) * batch_size)
    print("------------")

    tokenizer_layer = tf.keras.layers.TextVectorization(
        standardize='lower_and_strip_punctuation',
        split='whitespace',
        max_tokens=vocab_size,
        output_sequence_length=max_length,
    )

    # On entraine le tokenizer sur l'ensemble de données d'entraînement
    tokenizer_layer.adapt(str_train_ds.map(lambda text, label: text))
    print("Fin de l'entraînement du tokenizer")
    print("------------")


    # Préparation des ensembles de données : 
    def tokenize_text(text, label):
        text = tf.expand_dims(text, -1) # Explication -1 -> tf.data.Dataset -> "map"
        res = tokenizer_layer(text)
        
        return res, label

    train_ds = str_train_ds.map(tokenize_text)
    val_ds = str_val_ds.map(tokenize_text)
    test_ds = str_test_ds.map(tokenize_text)

    AUTOTUNE = tf.data.AUTOTUNE

    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
    test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

    print("Fin de la préparation des bases de donneés")
    print(len(train_ds) * batch_size, len(val_ds) * batch_size, len(test_ds) * batch_size)
    print("-----------")

    return tokenizer_layer, train_ds, val_ds, test_ds
    

La transformation dataframe -> ensemble de données tf (avec batch) semble avoir fonctionnée. On remarque cependant un petit problème au niveau de __l'encodage utf-8__ (dû à la présence d'accents dans la langue française) :

### Définition du modèle :

In [10]:
embedding_dim = 150 # Dimension avec laquelle on représente nos jetons

def build_network(vocab_size, embedding_dim, summary):

  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1),  # activation=None de base, la sortie n'est donc pas normalisée
  ])

  model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=3e-4),
    metrics=tf.metrics.BinaryAccuracy(threshold=0.0)
  )

  if summary:
      model.summary()

  return model

### Entraînement

In [11]:
%load_ext tensorboard

In [13]:
PATH = "/Users/erwan/Programmes/Stage/dlexperiments/Erwan/Text_Classification/datasets/Tweeter/french_tweets.csv"

df = pd.read_csv(PATH)
i = 6

In [16]:
DS_SIZE = 10000 # Ensemble de données entier : 1526724
BATCH_SIZE = 200
VOCAB_SIZE = 5000
EMBEDDING_DIM = 200
MAX_LENGTH = EMBEDDING_DIM

EPOCHS = 5

In [17]:
i += 1
num_run = "0" + str(i)

log_dir = "logs/run/" + num_run
log_model = 'logs/run/' + num_run + '/models'

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, 
    histogram_freq=1
)


model_checkpoint_callbacks = tf.keras.callbacks.ModelCheckpoint(
    filepath=log_model,
    save_weights_only=True,
    monitor='val_binary_accuracy',
    mode='max',
    save_best_only=True,
    initial_value_threshold=0.60,
    verbose=1
)s

early_stopping_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor='val_binary_accuracy',
    min_delta=0.001,
    patience=5,
    verbose=1,
    mode='auto',
    baseline=None,
    restore_best_weights=False
)

In [18]:
tokenizer, train_ds, val_ds, test_ds = build_datasets(
    df=df,
    ds_size=DS_SIZE,
    batch_size=BATCH_SIZE,
    vocab_size=VOCAB_SIZE,
    max_length=MAX_LENGTH,
)

model = build_network(VOCAB_SIZE, EMBEDDING_DIM, True)

------------
 Ensemble de données créé, taille : 10000
------------
 Taille des ensembles de données : 7000, 1500
Fin du chargement des bases de données
7000 1600 1600
------------
Fin de l'entraînement du tokenizer
------------
Fin de la préparation des bases de donneés
7000 1600 1600
-----------
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 200)         1000000   
                                                                 
 dropout (Dropout)           (None, None, 200)         0         
                                                                 
 global_average_pooling1d (G  (None, 200)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                     

In [19]:
print(f"Entraînement pour un ensemble de données de {DS_SIZE} exemples dont {len(train_ds) * BATCH_SIZE} pour l'entraînement")
print()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[tensorboard_callback, model_checkpoint_callbacks, early_stopping_callbacks],
)

print(history.history)

Entraînement pour un ensemble de données de 10000 exemples dont 7000 pour l'entraînement

Epoch 1/5
35/35 [==============================] - ETA: 0s - loss: 0.6929 - binary_accuracy: 0.5079
Epoch 1: val_binary_accuracy did not improve from 0.60000
35/35 [==============================] - 3s 71ms/step - loss: 0.6929 - binary_accuracy: 0.5079 - val_loss: 0.6930 - val_binary_accuracy: 0.4920
Epoch 2/5
35/35 [==============================] - ETA: 0s - loss: 0.6926 - binary_accuracy: 0.5283
Epoch 2: val_binary_accuracy did not improve from 0.60000
35/35 [==============================] - 2s 52ms/step - loss: 0.6926 - binary_accuracy: 0.5283 - val_loss: 0.6927 - val_binary_accuracy: 0.4920
Epoch 3/5
34/35 [============================>.] - ETA: 0s - loss: 0.6922 - binary_accuracy: 0.5381
Epoch 3: val_binary_accuracy did not improve from 0.60000
35/35 [==============================] - 2s 54ms/step - loss: 0.6922 - binary_accuracy: 0.5384 - val_loss: 0.6924 - val_binary_accuracy: 0.4940
Epoc

In [203]:
loss, accuracy = model.evaluate(test_ds)

print(f"Erreur : {loss}")
print(f"Précision : {accuracy}")

79/79 [==============================] - 0s 1ms/step - loss: 0.7834 - binary_accuracy: 0.6924
Erreur : 0.783367395401001
Précision: 0.6923999786376953


In [ ]:
export_model = tf.keras.Sequential([
    tokenizer,
    model
]) #Ajouter une sigmoïde ?

export_model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
    optimizer="adam",
    metrics=['accuracy'],
)

In [ ]:
export_model.save('logs/export_models/model' + str(i))